## Chapter 3

## Overview of Machine Learning and Deep Learning Concepts

This chapter is focused on exploring the realm of machine learning and deep learning algorithms. 


In [ ]:
# import all the libraries required for this chapter
# Machine Learning Libraries: scikit-learn, keras and tensorflow

# setting seed for model reproducibility
!pip install scikeras
seed_value = 42
import os
os.environ['PYTHONHASHSEED'] = str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)
import pandas as pd

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize, Normalizer, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn import metrics
from sklearn.compose import ColumnTransformer
import sklearn.cluster as cluster
import lightgbm as lgb
import xgboost as xgb

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from scikeras.wrappers import KerasRegressor, KerasClassifier
from tensorflow.keras.utils import to_categorical as np_utils

# plotting libraries
import matplotlib as mpl
import matplotlib.style
import seaborn as sns  # visualization
import matplotlib.pyplot as plt
# formatting for decimal places
pd.set_option("display.float_format", "{:.2f}".format)
plt.style.use("seaborn-v0_8-white")
sns.set_style("white")
import warnings
warnings.filterwarnings('ignore')

## --------   For Google CoLab Only   --------
## Run this cell to download the dataset automatically. Skip if you already have 'Merged_Data.csv'.

In [ ]:
!pip install gdown --quiet
import gdown


url = ' https://drive.google.com/uc?id=1189ESk9vlZuklYHLhIlq_tJ61TSwMryY'
gdown.download(url, 'Merged_Data.csv', quiet=False)

data = pd.read_csv('Merged_Data.csv')
data.info()

## Run this cell if you already have 'Merged_Data.csv'.

In [ ]:
# import the .csv file as a dataframe - Raw Data File
data = pd.read_csv('./data/Merged_Data.csv')
data.info()

## Continue...

In [ ]:
# Basic Exploratory Data Analysis and Data Cleaning
print(data.head(10))
print('Shape of Dataset (rows,columns):', data.shape)

Until this point there is no data cleaning being performed. The objective of the next few sections is to wrangle the data and prepare it for the machine learning model stage. The current dataset consists of petrophysical properties from two different wells in Volve field. A majority of columns have float datatype except Well Name and Lithotype. In the case of regression problems, only quantitative columns would be considered, whereas in the case of classification, 'Lithotype' would be the response variable. 

In [ ]:
# Find number of empty/NA values in each column
data.isna().sum().plot(kind="bar")

In [ ]:
# Descriptive Statistics
data.describe(include="all").T

In [ ]:
# Number of unique values
data.nunique().plot(kind="bar")

In [ ]:
# Number of unique values
data.nunique().plot(kind="bar")

In [ ]:
# Data Cleaning - How many columns have Null or value=0
data[data == 0].count(axis=0).plot(kind='bar')

In [ ]:
data.dropna(inplace=True)
print('Shape of Dataset (rows,columns):', data.shape)

In [ ]:
# Histogram - Distributions of continuous variables - only key features are shown here
# sns.set(color_codes=True)
f, axes = plt.subplots(3, 3, figsize=(15, 12), sharex=False)
sns.distplot(data["DEPTH (M)"], color="blue", ax=axes[0, 0])
sns.distplot(data["BVW (V/V)"], color="olive", ax=axes[0, 1])
sns.distplot(data["KLOGH (MD)"], color="teal", ax=axes[0, 2])
sns.distplot(data["PHIF (V/V)"], color="blue", ax=axes[1, 0])
sns.distplot(data["RHOFL (G/CM3)"], color="olive", ax=axes[1, 1])
sns.distplot(data["RW (OHMM)"], color="teal", ax=axes[1, 2])
sns.distplot(data["SW (V/V)"], color="blue", ax=axes[2, 0])
sns.distplot(data["VSH (V/V)"], color="olive", ax=axes[2, 1])
sns.distplot(data["TEMP (DEGC)"], color="teal", ax=axes[2, 2])

In [ ]:
# plotting some categorical variables
sns.catplot(x="COAL_FLAG (UNITLESS)", y="KLOGH (MD)", data=data)
# sns.catplot(x="SAND_FLAG (UNITLESS)", y="KLOGH (MD)", data=data)
# sns.catplot(x="RHOFL (G/CM3)", y="KLOGH (MD)", data=data)

In [ ]:
# data['CARB_FLAG (UNITLESS)'].value_counts().plot(kind='bar')
# data['COAL_FLAG (UNITLESS)'].value_counts().plot(kind='bar')
data['SAND_FLAG (UNITLESS)'].value_counts().plot(kind='bar')

In [ ]:
# Box Plots - Useful Tool to detect outliers
f, axes = plt.subplots(3, 3, figsize=(12, 15), sharex=False)
sns.boxplot(data["DEPTH (M)"], color="blue", ax=axes[0, 0])
sns.boxplot(data["BVW (V/V)"], color="olive", ax=axes[0, 1])
sns.boxplot(data["KLOGH (MD)"], color="teal", ax=axes[0, 2])
sns.boxplot(data["PHIF (V/V)"], color="blue", ax=axes[1, 0])
sns.boxplot(data["RHOFL (G/CM3)"], color="olive", ax=axes[1, 1])
sns.boxplot(data["RW (OHMM)"], color="teal", ax=axes[1, 2])
sns.boxplot(data["SW (V/V)"], color="blue", ax=axes[2, 0])
sns.boxplot(data["VSH (V/V)"], color="olive", ax=axes[2, 1])
sns.boxplot(data["TEMP (DEGC)"], color="teal", ax=axes[2, 2])

In [ ]:
# heatmap to visualize any collinearlity between variables
plt.figure(figsize=(12, 10))
# Colormap definition
cmap = sns.diverging_palette(220, 10, as_cmap=True)
numeric_data = data.iloc[:, 1:-1]
sns.heatmap(numeric_data.corr(), annot=True, fmt='.1g', vmin=-
            1, vmax=1, center=0, cmap=cmap, square=True)
# matplotlib issue with truncation of top and bottom row
b, t = plt.ylim()
b += 0.5  # Add 0.5 to the bottom
t -= 0.5  # Subtract 0.5 from the top
plt.ylim(b, t)  # update the ylim(bottom, top) values

In [ ]:
# An example of plot to check collinearity between certain variables
# looking into the correlation between Spacing and NN Spacing - All Zone/Same Zone
sns.regplot(x='PHIF (V/V)', y='KLOGH (MD)', data=data)
plt.xticks(rotation=90)

In [ ]:
data.info()

In [ ]:
# separate categorical columns
columns_categ = ['WELL NAME', 'LITHOTYPE']
data_cont = data.drop(data[columns_categ], axis=1)  # continous data
# move response variable to end of dataframe
data_cont = data_cont[[col for col in data_cont.columns if col != 'KLOGH (MD)'] + ['KLOGH (MD)']]
data_cont.info()

In [ ]:
# Splitting the dataset
X = data_cont.iloc[:, :-1]
y = data_cont.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print('Shape of Training X:', X_train.shape)
print('Shape of Training y:', y_train.shape)
print('Shape of Test X:', X_test.shape)
print('Shape of Test y:', y_test.shape)

## Regression Algorithms

In [ ]:
# defining some functions for regression
# Regression

def reg_metrics(test, pred):
    '''Function returns basic metrics for regression models'''
    print('Mean Absolute Error:', metrics.mean_absolute_error(test, pred))
    print('Mean Squared Error:', metrics.mean_squared_error(test, pred))
    print('Root Mean Squared Error:', np.sqrt(
        metrics.mean_squared_error(test, pred)))
    print('R Squared:', (metrics.r2_score(test, pred)))

def reg_plot(test, pred):
    '''Function returns a regression plot, in the form of a scatter plot'''
    sns.regplot(x=test, y=pred, scatter_kws={
                "color": "black"}, line_kws={"color": "red"})
    plt.xlim(0, max(test))
    plt.ylim(0, max(pred))
    plt.title('Regression Results')
    plt.xlabel('Test Data')
    plt.ylabel('Model Prediction')
    plt.show()

def scatter_plot_comparison(test, pred):
    '''Function returns a comparison between test data and predictions, in the form of line plot'''
    sns.scatterplot(x=test.index, y=test.values, color='red', label='Test data')
    sns.scatterplot(x=test.index, y=pred, color='blue', label='Predicted data')
    plt.title('Prediction')
    plt.xlabel('Observations')
    plt.ylabel('Test and Predicted Values')
    plt.legend()
    plt.show()

def line_plot_comparison(test, pred):
    '''Function returns a comparison between test data and predictions, in the form of line plot'''
    plt.plot(test.values, color='red', label='Test data')
    plt.plot(pred, color='blue', label='Predicted data')
    plt.title('Prediction')
    plt.xlabel('Observations')
    plt.ylabel('Test and Predicted Values')
    plt.legend()
    plt.show()

In [ ]:
# 1. MLR - Multi Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)  # training the algorithm
print(lin_reg.intercept_)  # intercept
print(lin_reg.coef_)  # coefficients
# Prediction on test data
y_pred_lin = lin_reg.predict(X_test)
# Regression Plot - Linear Regression
reg_plot(y_test, y_pred_lin)
scatter_plot_comparison(y_test, y_pred_lin)
# Metrics for Linear Regression
reg_metrics(y_test, y_pred_lin)

In [ ]:
# 2. Support Vector Regression (SVR)
# gaussian kernel selected due to non-linearity in dataset
# Radial basis function - kernel is shown here, other available kernels include 'linear, sigmoid and polynomial'
svr_reg = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_reg.fit(X_train, y_train)
# Prediction on test data
y_pred_svr = svr_reg.predict(X_test)
# Regression Plot - SVR
reg_plot(y_test, y_pred_svr)
scatter_plot_comparison(y_test, y_pred_svr)
# Metrics for SVR
reg_metrics(y_test, y_pred_svr)

In [ ]:
# 3. Decision Tree - Regression
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train, y_train)
# Prediction on test data
y_pred_dt = dt_reg.predict(X_test)
# Regression Plot - Decision Tree
reg_plot(y_test, y_pred_dt)
scatter_plot_comparison(y_test, y_pred_dt)
# Metrics for Decision Tree
reg_metrics(y_test, y_pred_dt)

In [ ]:
# 4. Random Forest - Regression
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train, y_train)
# Prediction on test data
y_pred_rf = rf_reg.predict(X_test)
# Regression Plot - Random Forest
reg_plot(y_test, y_pred_rf)
scatter_plot_comparison(y_test, y_pred_rf)
# Metrics for Random Forest
reg_metrics(y_test, y_pred_rf)

In [ ]:
# 5. XGBoost - Regression
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)
# Prediction on test data
y_pred_xgb = xgb_reg.predict(X_test)
# Regression Plot - XGBoost
reg_plot(y_test, y_pred_xgb)
scatter_plot_comparison(y_test, y_pred_xgb)
# Metrics for XGBoost
reg_metrics(y_test, y_pred_xgb)

In [ ]:
# 6. Artificial Neural Network
# scaling the dataset
std_scalar_X = StandardScaler()
std_scalar_Y = StandardScaler()
X_train_scaled = std_scalar_X.fit_transform(X_train)
X_test_scaled = std_scalar_Y.fit_transform(X_test)

def build_model():
    ann_reg = keras.Sequential([
        layers.Dense(32, activation='relu', input_shape=[len(X_train.keys())]),
        layers.Dense(32, activation='relu'),
        layers.Dense(1) ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)
    ann_reg.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return ann_reg

ann_reg = build_model()
ann_reg.summary()

history = ann_reg.fit(X_train_scaled, y_train.values, epochs=1000, validation_split=0.25, verbose=1)
print(history.history.keys())
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('Model MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')

y_pred_ann = ann_reg.predict(X_test_scaled).flatten()
# Regression Plot - ANN
reg_plot(y_test, y_pred_ann)
scatter_plot_comparison(y_test, y_pred_ann)
# Metrics for ANN
reg_metrics(y_test, y_pred_ann)

## Classification Algorithms

In [ ]:
# The response variable in the classification is 'Lithotype' with all remaining features as input'
data['LITHOTYPE'].value_counts().plot(kind='bar')
plt.show()
count_categ = data['LITHOTYPE'].value_counts()
print(count_categ)

In [ ]:
# As observed multiple categories are present, however 'other' won't be considered and
# only the categories with frequency of more than 200 will be included, i.e. 3 categories
data_categ = data[data['LITHOTYPE'].isin(count_categ[count_categ > 200].index)]
data_categ = data_categ[data_categ['LITHOTYPE'] != 'OTHER']
data_categ['LITHOTYPE'].value_counts()
data_categ['LITHOTYPE'].value_counts().plot(kind='bar')

In [ ]:
data_categ.drop('WELL NAME', axis=1, inplace=True)
data_categ.info()
data_categ.head(10)

In [ ]:
X = data_categ.iloc[:, :-1]
y = data_categ.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print('Shape of Training X:', X_train.shape)
print('Shape of Training y:', y_train.shape)
print('Shape of Test X:', X_test.shape)
print('Shape of Test y:', y_test.shape)

In [ ]:
# defining function for classification metrics
def clf_metrics(test, pred):
    '''Function returns basic metrics for classification models'''
    print('Classification Accuracy Score:', accuracy_score(test, pred))
    print('Confusion Matrix: \n', confusion_matrix(test, pred))
    print('Classification Report: \n', classification_report(test, pred))

In [ ]:
# 1. Logistic Regression
clf_logreg = LogisticRegression()
clf_logreg.fit(X_train, y_train)
# Prediction on test data
y_pred_logreg = clf_logreg.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_logreg)

In [ ]:
# 2. SVC
clf_svc = SVC()
clf_svc.fit(X_train, y_train)
# Prediction on test data
y_pred_svc = clf_svc.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_svc)

In [ ]:
# 3. Decision Tree Classifier
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_train, y_train)
# Prediction on test data
y_pred_dt = clf_dt.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_dt)

In [ ]:
# 4. Random Forest Classifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
# Prediction on test data
y_pred_rf = clf_rf.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_rf)

In [ ]:
# 5. KNN
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)
# Prediction on test data
y_pred_knn = clf_knn.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_knn)

In [ ]:
# 6. GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_train, y_train)
# Prediction on test data
y_pred_gnb = clf_gnb.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_gnb)

In [ ]:
# 7. LDA
clf_lda = LinearDiscriminantAnalysis()
clf_lda.fit(X_train, y_train)
# Prediction on test data
y_pred_lda = clf_lda.predict(X_test)
# Accuracy Metrics
clf_metrics(y_test, y_pred_lda)

In [ ]:
# 8. ANN

dataset = data_categ.values
X = dataset[:,0:12].astype(float)
Y = dataset[:,13]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# creating a dummy variable
dummy_y = np_utils.to_categorical(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.25, random_state=42)

# define baseline model
def baseline_model():
    # create model
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, input_dim=12, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

ann_clf = baseline_model()
ann_clf.summary()

ann_classifier = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)
# Cross Validation Score
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(ann_classifier, X, dummy_y, cv=kfold)

In [ ]:
print("Base Model Accuracy (Standard Deviation): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Unsupervised Learning

In [ ]:
# original dataset with all features
data.info()
data.head(10)

In [ ]:
sns.pairplot(data)

In [ ]:
X = data.drop(['WELL NAME', 'LITHOTYPE'], axis=1)
X.info()

In [ ]:
# KMeans Clustering

# finding the optimum number of clusters in the dataset
clusters = []
for i in range(1, 13):
    km = cluster.KMeans(n_clusters=i).fit(X)
    clusters.append(km.inertia_)

# finding the number of clusters
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(1, 13)), y=clusters, ax=ax)
ax.set_title('Finding Optimum Number of Clusters')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

In [ ]:
#  Visual Plot - 3 cluster
km3 = cluster.KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42).fit(X)
X['Labels'] = km3.labels_
X['Labels'].value_counts()
sns.scatterplot(x=X['PHIF (V/V)'], y=X['KLOGH (MD)'], hue=X['Labels'], palette=sns.color_palette('hls', X['Labels'].nunique()))
plt.title('KMeans with 3 Clusters')

In [ ]:
# PCA 

data_categ.info()
dataset = data_categ.values
# One hot encoding example instead of LabelEncoder used earlier
# Encoding - Convert categorical variables to numerical form, to execute machine learning algorithms
X = data_categ.drop('LITHOTYPE', axis=1)
y = data_categ['LITHOTYPE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
# 4 components can explain around 86.6% variablility 
# only 4 components will be used to test classification 
pca_4comp = PCA(n_components=4)
X_train_4comp = pca_4comp.fit_transform(X_train)
X_test_4comp = pca_4comp.transform(X_test)

In [ ]:
# Random Forest with entire data
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
# Random Forest with only 4 components
classifier_4comp = RandomForestClassifier(random_state=42)
classifier_4comp.fit(X_train_4comp, y_train)
# Predicting the Test set results
y_pred_4comp= classifier_4comp.predict(X_test_4comp)
print(confusion_matrix(y_test, y_pred_4comp))
print(accuracy_score(y_test, y_pred_4comp))